# Discrete-Time Models of Belief Dynamics


### What are we modeling?

* A **closed, well-mixed population** whose members can hold one of a few opinions.  
* In every **time step** $t\!\to\!t{+}1$ people may **switch camp** according to simple, repeatable rules.  
* Goal: see whether the population ends up in **consensus**, **polarisation**, or a **mixed balance**.

Core state variables  

| Symbol | Meaning | Range | Mass balance |
|--------|---------|-------|--------------|
| $p_{b,t}$ | believers | $[0,1]$ | $\displaystyle p_{b,t}+p_{n,t}+p_{a,t}=1$ |
| $p_{n,t}$ | non-believers / skeptics | $[0,1]$ | |
| $p_{a,t}$ | agnostics / undecided | $[0,1]$ | |

Baseline conversion (works in **every** variant)  

$
\Delta_{x\!\to y}\in[0,1]
\quad\text{= per-step probability that a single }x\text{-agent becomes }y.
$

Example for the three-state baseline:  

$
\begin{aligned}
p_{b,t+1} &= p_{b,t}
           +\Delta_{n\to b}\,p_{n,t}
           +\Delta_{a\to b}\,p_{a,t}
           -\Delta_{b\to n}\,p_{b,t}
           -\Delta_{b\to a}\,p_{b,t},\\[4pt]
p_{n,t+1} &= p_{n,t}
           +\Delta_{b\to n}\,p_{b,t}
           +\Delta_{a\to n}\,p_{a,t}
           -\Delta_{n\to b}\,p_{n,t}
           -\Delta_{n\to a}\,p_{n,t},\\[4pt]
p_{a,t+1} &= 1 - p_{b,t+1} - p_{n,t+1}.
\end{aligned}
$

We assume  

- **Baseline conversion rates**:  
  $p_{nb}$ non → believer per unit “influence,”  
  $p_{bn}$ believer → non per unit “influence.”  

- **Social / campaign intensity**:  
  $r\ge0$ (scales peer-pressure or campaign effects) and  
  $\varepsilon>0$ (small regularizer to avoid division by zero).  

Iterate each model for $t=1,\dots,T$.

---

## Putting it together  

* **Choose a variant** (baseline only, social-pressure, inverse-popularity…).  
* **Set parameters** $(\Delta, r, \varepsilon)$ and **initial shares**.  
* **Iterate** for $t=0,\dots,T$.  
* **Plot** $p_{b},p_{n},p_{a}$ to watch consensus, oscillations, or stalemates emerge.

---

**Takeaway:**  
Even with *tiny* rule sets, discrete-time belief models can reproduce a surprising range of collective behaviours. Tweaking just one term lets you switch from “majority wins” to “support the minority,” giving a flexible playground for classroom demos or quick policy experiments.


## 1. Minimal Two-State Model

No undecided pool. Update:  
$p_{b,t}=p_{b,t-1}+p_{nb}\bigl(1-p_{b,t-1}\bigr)-p_{bn}p_{b,t-1},\quad p_{n,t}=1-p_{b,t}.$  

Flows: $n\to b$ is $p_{nb}(1-p_{b,t-1})$;  
$b\to n$ is $p_{bn}p_{b,t-1}$.  

Long-run:  
* If $p_{nb}>p_{bn}$ then $p_{b,t}\to1$.  
* If $p_{nb}<p_{bn}$ then $p_{b,t}\to0$.  
* If $p_{nb}=p_{bn}$ then $p_{b,t}=p_{b,0}$.

and $p_{n,t}=1-p_{b,t}$.

  $p_{nb}$ answers: ‘what share of non-believers flip today?’

  $p_{bn}$ answers: ‘what share of believers defect today?’

If $p_{nb}>p_{bn}$ the mathematics predicts eventual full belief; if the reverse, disbelief wins; equality locks the initial split in place.”



In [ ]:
# Imports
import matplotlib.pyplot as plt

In [ ]:
# Parameters

decay_rate: float = 0.0                  # Rate at which believers lose belief
adoption_rate: float = 0.0               # Rate at which non-believers adopt belief
steps: int = 0                           # Number of time steps
believers: float = 0.0                   # Current proportion of believers

believer_trend: list[float] = []         # History of believer proportions
non_believer_trend: list[float] = []     # History of non-believer proportions


def initialize(belief_gain: float, belief_loss: float, step_count: int, initial: float) -> None:
    global decay_rate, adoption_rate, steps, believers, believer_trend, non_believer_trend

    decay_rate = belief_loss
    adoption_rate = belief_gain
    steps = step_count
    believers = initial

    believer_trend.clear()
    non_believer_trend.clear()

    believer_trend.append(believers)
    non_believer_trend.append(1 - believers)


def observe(current: float) -> None:
    global believer_trend, non_believer_trend

    believer_trend.append(current)
    non_believer_trend.append(1 - current)


def update() -> None:
    global believers, adoption_rate, decay_rate

    believers = (
        believers
        + adoption_rate * (1 - believers)
        - decay_rate * believers
    )


# Initialize with chosen values
initialize(belief_gain=0.2, belief_loss=0.1, step_count=30, initial=0.3)

# Run simulation
for t in range(steps):
    update()
    observe(believers)

# Plot results
plt.plot(believer_trend, 'r-', label='Believers')
plt.plot(non_believer_trend, 'b--', label='Non-Believers')

plt.title('Belief Conversion Over Time')
plt.xlabel('Time step')
plt.ylabel('Proportion')
plt.legend()
plt.grid(True)
plt.show()


In this model we see that although our believers start low, they quickly climb up as we set our adoption rate higher that our rate of loss. Eventually, we hit our state of equilibrium equal to the imblance of our rates.

## 2. Belief Dynamics With a Neutral Group

Let  

* $p_{b,t}$ = fraction of **believers** at step $t$ (`believers`)  
* $p_{s,t}$ = fraction of **skeptics** (`skeptics`)  
* $p_{a,t}$ = fraction of **agnostics** (`agnostics`)  

$b_t = b_{t-1}+ p_{s->b}s_{t-1}+p_{a->b}a_{t-1} - (p_{b->s} + p_{b->a})b_{t-1}$,

 $s_t = s_{t-1}+ p_{b->s}b_{t-1}+p_{a->s}a_{t-1} - (p_{s->b} + p_{s->s})s_{t-1}$,

 $a_t = a_{t-1}+ p_{b->a}b_{t-1}+p_{s->a}s_{t-1} - (p_{a->b} + p_{a->s})a_{t-1}$

with $p_{b,t}+p_{s,t}+p_{a,t}=1$.

### Conversion-rate parameters  

| Python name | Symbol | Meaning (per step) |
|-------------|--------|--------------------|
| `believer_to_skeptic` | $p_{b\to s}$ | believer → skeptic |
| `believer_to_agnostic` | $p_{b\to a}$ | believer → agnostic |
| `skeptic_to_believer` | $p_{s\to b}$ | skeptic → believer |
| `skeptic_to_agnostic` | $p_{s\to a}$ | skeptic → agnostic |
| `agnostic_to_believer` | $p_{a\to b}$ | agnostic → believer |
| `agnostic_to_skeptic`  | $p_{a\to s}$ | agnostic → skeptic |

All rates lie in $[0,1]$ and can be chosen independently.







In [ ]:
# Parameters

steps: int = 0                           # Number of time steps

believers: float = 0.0                   # Current proportion of believers
skeptics: float = 0.0
agnostics: float = 0.0

# Conversion rates
believer_to_skeptic:float = 0.0
believer_to_agnostic:float = 0.0
skeptic_to_believer:float = 0.0
skeptic_to_agnostic:float = 0.0
agnostic_to_believer:float = 0.0
agnostic_to_skeptic:float = 0.0

# Lists to hold state in time for proportions
believer_trend: list[float] = []          # History of believer proportions
skeptic_trend: list[float] = []           # History of skeptic proportions
agnostic_trend: list[float] = []          # History of agnostic proportions

# Helper function for setting initial population proportions
def set_initial_beliefs(believe: float, skeptical: float, agnostic: float) -> None:
    global believers, skeptics, agnostics

    believers = believe
    skeptics = skeptical
    agnostics = agnostic

# Helper function for seting conversion rates between
def set_conversion_rates(b_to_s: float, b_to_a: float, s_to_b: float, s_to_a: float, a_to_b: float, a_to_s: float) -> None:
    global believer_to_skeptic, believer_to_agnostic
    global skeptic_to_believer, skeptic_to_agnostic
    global agnostic_to_believer, agnostic_to_skeptic

    believer_to_skeptic = b_to_s
    believer_to_agnostic = b_to_a
    skeptic_to_believer = s_to_b
    skeptic_to_agnostic = s_to_a
    agnostic_to_believer = a_to_b
    agnostic_to_skeptic = a_to_s

# Initialization function that calls both helper functions
def initialize(step_count: int) -> None:
    global steps
    global believers, agnostics, skeptics
    global believer_trend, skeptic_trend, agnostic_trend

    set_initial_beliefs(0.4, 0.2, 0.4)
    set_conversion_rates(0.6, 0.2, 0.4, 0.1, 0.2, 0.1)
    steps = step_count

    believer_trend.clear()
    skeptic_trend.clear()
    agnostic_trend.clear()

    believer_trend.append(believers)
    skeptic_trend.append(skeptics)
    agnostic_trend.append(agnostics)

# observation functions that adds the current state of our variables to thie respective lists
def observe() -> None:
    global believers, skeptics, agnostics
    global believer_trend, skeptic_trend, agnostic_trend

    believer_trend.append(believers)
    skeptic_trend.append(skeptics)
    agnostic_trend.append(agnostics)

# Updates future proportions based on current proportions and conversion rates
def update() -> None:
    global believers, skeptics, agnostics
    global believer_to_skeptic, believer_to_agnostic
    global skeptic_to_believer, skeptic_to_agnostic
    global agnostic_to_believer, agnostic_to_skeptic

    updated_believers = (
        believers
        + skeptic_to_believer * skeptics
        + agnostic_to_believer * agnostics
        - believer_to_skeptic * believers
        - believer_to_agnostic * believers
    )
    updated_skeptics = (
        skeptics
        + believer_to_skeptic * believers
        + agnostic_to_skeptic * agnostics
        - skeptic_to_believer * skeptics
        - skeptic_to_agnostic * skeptics
    )

    # Agnostics are updated as a function of the new state of believers and skeptics
    # This to ensure the total stays 1 without normailzation
    updated_agnostics = (1 - updated_believers) - updated_skeptics

    # Set new proportions
    believers = updated_believers
    skeptics = updated_skeptics
    agnostics = updated_agnostics

# Initialize with chosen values
initialize(step_count=30,)

# Run simulation
for t in range(steps):
    update()
    observe()

# Plot results in 2d
plt.plot(believer_trend, 'r-', label='Believers')
plt.plot(skeptic_trend, 'b--', label='Skeptics')
plt.plot(agnostic_trend, 'g-.', label='Agnostics')
plt.title('Belief Dynamics with Neutral Group')
plt.xlabel('Time step')
plt.ylabel('Proportion')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.show()

## Belief-Dynamics with Social Pressure  
---

### State Variables
$ p_{b,t},\;p_{n,t},\;p_{a,t} $  

* **Definition:** Fractions of believers (b), non-believers / skeptics (n), and agnostics (a) at time t.  
* **Constraint:** $ p_{b,t}+p_{n,t}+p_{a,t}=1 $ keeps the population size fixed.  
* **Key point for audience:** Everything that follows simply redistributes people among these three boxes.

---

### Baseline Conversions  

$ \Delta_{b\to n},\;\Delta_{b\to a},\;\Delta_{n\to b},\;\Delta_{n\to a},\;\Delta_{a\to b},\;\Delta{a\to n}\in[0,1] $  

* **What they are:** Constant per-step probabilities for spontaneous switching (e.g. media exposure, personal doubts).  
* **Why we need them:** They act even when no social-pressure term is present, preventing the model from freezing.

---

### Social-Pressure Fluxes (2-state core idea)  

$ \displaystyle \Delta_{n\to b}(t)=r\,\max\!\bigl(p_{n,t-1}-p_{b,t-1},0\bigr)\,p_{n,t-1} $  

$ \displaystyle \Delta_{b\to n}(t)=r\,\max\!\bigl(p_{b,t-1}-p_{n,t-1},0\bigr)\,p_{b,t-1} $  

* **$r$ (campaign intensity):** Master knob - higher $r$ means stronger crowd influence.  
* **$ \max(\dots,0)$:** Only the *larger* camp exerts pressure; the minority does not.  
* **Multiplication by the source fraction ($p_{n,t-1}$ or $p_{b,t-1}$):** More potential switchers ⇒ larger flux.

---

### What to Demonstrate in Plots  

* **Consensus:** Show runs where $p_{b}\to1$ or $p_{n}\to1$ by adjusting $r$.  
* **Mixed equilibrium:** Tune the $\Delta$'s so none of the three states vanishes.  
* **Sensitivity:** Vary $\varepsilon$ - explain it only affects extreme cases, not ordinary dynamics.


In [ ]:
# Parameters

steps: int = 0                           # Number of time steps

social_pressure_strength: float = 0.5    # r in the equations


believers: float = 0.0                   # Current proportion of believers
skeptics: float = 0.0
agnostics: float = 0.0

# Conversion rates
believer_to_skeptic:float = 0.0
believer_to_agnostic:float = 0.0
skeptic_to_believer:float = 0.0
skeptic_to_agnostic:float = 0.0
agnostic_to_believer:float = 0.0
agnostic_to_skeptic:float = 0.0

# Lists to hold state in time for proportions
believer_trend: list[float] = []          # History of believer proportions
skeptic_trend: list[float] = []           # History of skeptic proportions
agnostic_trend: list[float] = []          # History of agnostic proportions

# Helper function for setting initial population proportions
def set_initial_beliefs(believe: float, skeptical: float, agnostic: float) -> None:
    global believers, skeptics, agnostics

    believers = believe
    skeptics = skeptical
    agnostics = agnostic

# Helper function for seting conversion rates between
def set_conversion_rates(b_to_s: float, b_to_a: float, s_to_b: float, s_to_a: float, a_to_b: float, a_to_s: float) -> None:
    global believer_to_skeptic, believer_to_agnostic
    global skeptic_to_believer, skeptic_to_agnostic
    global agnostic_to_believer, agnostic_to_skeptic

    believer_to_skeptic = b_to_s
    believer_to_agnostic = b_to_a
    skeptic_to_believer = s_to_b
    skeptic_to_agnostic = s_to_a
    agnostic_to_believer = a_to_b
    agnostic_to_skeptic = a_to_s

# Initialization function that calls both helper functions
def initialize(step_count: int) -> None:
    global steps
    global believers, agnostics, skeptics
    global believer_trend, skeptic_trend, agnostic_trend

    set_initial_beliefs(0.4, 0.2, 0.4)
    set_conversion_rates(0.6, 0.2, 0.4, 0.1, 0.2, 0.1)
    steps = step_count

    believer_trend.clear()
    skeptic_trend.clear()
    agnostic_trend.clear()

    believer_trend.append(believers)
    skeptic_trend.append(skeptics)
    agnostic_trend.append(agnostics)

# observation functions that adds the current state of our variables to thie respective lists
def observe() -> None:
    global believers, skeptics, agnostics
    global believer_trend, skeptic_trend, agnostic_trend

    believer_trend.append(believers)
    skeptic_trend.append(skeptics)
    agnostic_trend.append(agnostics)

# Updates future proportions based on current proportions and conversion rates
def update() -> None:
    global believers, skeptics, agnostics
    global believer_to_skeptic, believer_to_agnostic
    global skeptic_to_believer, skeptic_to_agnostic
    global agnostic_to_believer, agnostic_to_skeptic
    global social_pressure_strength

    delta_ab = social_pressure_strength * max(believers - skeptics, 0) * agnostics
    delta_an = social_pressure_strength * max(skeptics - believers, 0) * agnostics

    updated_believers = (
        believers
        + skeptic_to_believer * skeptics
        + agnostic_to_believer * agnostics
        + delta_ab
        - believer_to_skeptic * believers
        - believer_to_agnostic * believers
    )

    updated_skeptics = (
        skeptics
        + believer_to_skeptic * believers
        + agnostic_to_skeptic * agnostics
        + delta_an
        - skeptic_to_believer * skeptics
        - skeptic_to_agnostic * skeptics
    )

    updated_agnostics = (
        agnostics
        + believer_to_agnostic * believers
        + skeptic_to_agnostic * skeptics
        - agnostic_to_believer * agnostics
        - agnostic_to_skeptic * agnostics
        - delta_ab
        - delta_an
    )

    # normalize
    total = updated_believers + updated_skeptics + updated_agnostics
    believers = updated_believers / total
    skeptics = updated_skeptics / total
    agnostics = updated_agnostics / total

    # Set new proportions
    believers = updated_believers
    skeptics = updated_skeptics
    agnostics = updated_agnostics

# Initialize with chosen values
initialize(step_count=30,)

# Run simulation
for t in range(steps):
    update()
    observe()

# Plot results in 2d
plt.plot(believer_trend, 'r-', label='Believers')
plt.plot(skeptic_trend, 'b--', label='Skeptics')
plt.plot(agnostic_trend, 'g-.', label='Agnostics')
plt.title('Belief Dynamics with Social Pressure')
plt.xlabel('Time step')
plt.ylabel('Proportion')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.show()

## Belief Dynamics - Inverse-Popularity Switching  

---

### State Variables  

$ p_{b,t},\;p_{n,t},\;p_{a,t} $  

* **Believers** ($b$), **Skeptics** ($n$), **Agnostics** ($a$) at step $t$.  
* Mass balance: $ p_{b,t}+p_{n,t}+p_{a,t}=1 $.

---

### Model Parameters  

| Symbol | Meaning |
|--------|---------|
| $r$ | *campaign intensity* (`campaign_intensity` in the code) |
| $\varepsilon$ | tiny guard to avoid $1/0$ (`epsilon`) |
| `steps` | number of updates ⇒ $t=0,\dots,T$ |

---

### Baseline (spontaneous) conversion rates — **δ-notation**  

$ \delta_{b\to n},\;\delta_{b\to a},\;
  \delta_{n\to b},\;\delta_{n\to a},\;
  \delta_{a\to b},\;\delta_{a\to n}\in[0,1] $

*Direct word-of-mouth or media-driven switching that does **not** depend on who is winning.*

---

### Inverse-Popularity campaign fluxes (code lines `delta_ab`, `delta_an`)  

\begin{aligned}
\Delta_{a\to b}(t) &=
    \frac{r}{\,p_{n,t}+\varepsilon\,}\;
    p_{a,t}\,p_{b,t},\\[6pt]
\Delta_{a\to n}(t) &=
    \frac{r}{\,p_{b,t}+\varepsilon\,}\;
    p_{a,t}\,p_{n,t}.
\end{aligned}}

* When **skeptics** are tiny ($p_n\downarrow$) the first term grows, so agnostics slide toward believers.  
* When **believers** are tiny the second term dominates, boosting skeptics.  
* $\varepsilon\!(\approx10^{-6})$ guards against division by zero if a camp ever hits 0 %.

* **Red** = believers ($p_b$) **Blue dashed** = skeptics ($p_n$) **Green dot-dash** = agnostics ($p_a$).  
* Early oscillations show the *inverse-popularity* effect: whichever major camp is **smaller** gets rescued by agnostics.  
* Long-run outcome depends on $r$ and the $\delta$'s: you can land in believer consensus, skeptic consensus, or a mixed steady state.  
* **Live demo idea:** increase $r$ and watch the weaker camp recover faster (or overshoot).

**Notation tip:**  
 *δ* (lower-case) marks **fixed conversion constants**,  
 *Δ* (upper-case) marks **extra, time-dependent fluxes** driven by the campaign rule.  


In [ ]:
# Parameters

steps: int = 0                           # Number of time steps

campaign_intensity: float = 0.5    # r in the equations
epsilon: float = 1e-6              # small constant the avoids division by zero

believers: float = 0.0                   # Current proportion of believers
skeptics: float = 0.0
agnostics: float = 0.0

# Conversion rates
believer_to_skeptic:float = 0.0
believer_to_agnostic:float = 0.0
skeptic_to_believer:float = 0.0
skeptic_to_agnostic:float = 0.0
agnostic_to_believer:float = 0.0
agnostic_to_skeptic:float = 0.0

# Lists to hold state in time for proportions
believer_trend: list[float] = []          # History of believer proportions
skeptic_trend: list[float] = []           # History of skeptic proportions
agnostic_trend: list[float] = []          # History of agnostic proportions

# Helper function for setting initial population proportions
def set_initial_beliefs(believe: float, skeptical: float, agnostic: float) -> None:
    global believers, skeptics, agnostics

    believers = believe
    skeptics = skeptical
    agnostics = agnostic

# Helper function for seting conversion rates between
def set_conversion_rates(b_to_s: float, b_to_a: float, s_to_b: float, s_to_a: float, a_to_b: float, a_to_s: float) -> None:
    global believer_to_skeptic, believer_to_agnostic
    global skeptic_to_believer, skeptic_to_agnostic
    global agnostic_to_believer, agnostic_to_skeptic

    believer_to_skeptic = b_to_s
    believer_to_agnostic = b_to_a
    skeptic_to_believer = s_to_b
    skeptic_to_agnostic = s_to_a
    agnostic_to_believer = a_to_b
    agnostic_to_skeptic = a_to_s

# Initialization function that calls both helper functions
def initialize(step_count: int) -> None:
    global steps
    global believers, agnostics, skeptics
    global believer_trend, skeptic_trend, agnostic_trend

    set_initial_beliefs(0.4, 0.2, 0.4)
    set_conversion_rates(0.6, 0.2, 0.4, 0.1, 0.2, 0.1)
    steps = step_count

    believer_trend.clear()
    skeptic_trend.clear()
    agnostic_trend.clear()

    believer_trend.append(believers)
    skeptic_trend.append(skeptics)
    agnostic_trend.append(agnostics)

# observation functions that adds the current state of our variables to thie respective lists
def observe() -> None:
    global believers, skeptics, agnostics
    global believer_trend, skeptic_trend, agnostic_trend

    believer_trend.append(believers)
    skeptic_trend.append(skeptics)
    agnostic_trend.append(agnostics)

# Updates future proportions based on current proportions and conversion rates
def update() -> None:
    global believers, skeptics, agnostics
    global believer_to_skeptic, believer_to_agnostic
    global skeptic_to_believer, skeptic_to_agnostic
    global agnostic_to_believer, agnostic_to_skeptic
    global social_pressure_strength

    delta_ab = (campaign_intensity / (skeptics + epsilon)) * believers * agnostics
    delta_an = (campaign_intensity / (believers + epsilon)) * skeptics * agnostics

    delta_ab_trend.append(delta_ab)
    delta_an_trend.append(delta_an)


    updated_believers = (
        believers
        + skeptic_to_believer * skeptics
        + agnostic_to_believer * agnostics
        + delta_ab
        - believer_to_skeptic * believers
        - believer_to_agnostic * believers
    )

    updated_skeptics = (
        skeptics
        + believer_to_skeptic * believers
        + agnostic_to_skeptic * agnostics
        + delta_an
        - skeptic_to_believer * skeptics
        - skeptic_to_agnostic * skeptics
    )

    updated_agnostics = (
        agnostics
        + believer_to_agnostic * believers
        + skeptic_to_agnostic * skeptics
        - agnostic_to_believer * agnostics
        - agnostic_to_skeptic * agnostics
        - delta_ab
        - delta_an
    )

    # normalize
    total = updated_believers + updated_skeptics + updated_agnostics
    believers = updated_believers / total
    skeptics = updated_skeptics / total
    agnostics = updated_agnostics / total

    # Set new proportions
    believers = updated_believers
    skeptics = updated_skeptics
    agnostics = updated_agnostics

# Initialize with chosen values
initialize(step_count=30,)

delta_ab_trend = []
delta_an_trend = []

# Run simulation
for t in range(steps):
    update()
    observe()

# Plot results in 2d
plt.plot(believer_trend, 'r-', label='Believers')
plt.plot(skeptic_trend, 'b--', label='Skeptics')
plt.plot(agnostic_trend, 'g-.', label='Agnostics')
plt.title('Belief Dynamics with Inverse Social Pressure')
plt.xlabel('Time step')
plt.ylabel('Proportion')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.show()

## Conclusion - What Have we Learned From This?

* **Tiny rule sets → rich outcomes.**  
  A handful of **baseline switches** (δ-rates) and one extra **campaign term** (Δ-flux) already span consensus, polarisation, stalemates, and oscillations.

* **Two knobs steer everything.**  
  * **$r$** - amplifies / dampens social or media influence.  
  * **Variant choice** - majority-boost (social pressure) *vs.* minority-boost (inverse popularity).

* **Mass is king.**  
  Every update conserves $p_{b,t}+p_{n,t}+p_{a,t}=1$, so curves never wander outside $[0,1]$—a built-in reality check.

* **Visuals reveal the story.**  
  Side-by-side plots of $p_b$, $p_n$, $p_a$ instantly show which forces dominate and when tipping points occur.

* **Easy to extend.**  
  Add memory, time-varying rates, network structure, or more opinion states without rewriting the core bookkeeping.